In [30]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import warnings

warnings.filterwarnings('ignore')

In [31]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Oussama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
data = pd.read_csv("C:/Users/Oussama/Downloads/Twitter Sentiments.csv")
data

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [33]:
data.shape

(31962, 3)

In [34]:
data['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [39]:
ps = PorterStemmer()
corpus = []

for i in range(len(data)):
    #print(i)
    rev = re.sub("[^a-zA-Z]",' ', data['tweet'][i])
    rev = rev.lower() 
    rev = rev.split()
    rev = [ps.stem(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = ' '.join(rev)
    corpus.append(rev)

In [76]:
#corpus

# Convertir une collection de documents textuels en une matrice de tokens

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()

In [46]:
X.shape

(31962, 5000)

# Diviser le dataset en un train set et test set

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, data['label'], test_size=0.20, random_state = 0)


In [51]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((25569, 5000), (6393, 5000), (25569,), (6393,))

# Def le modèle

In [62]:
from sklearn.naive_bayes import MultinomialNB

In [63]:
mnb = MultinomialNB().fit(X_train, y_train)

# Test du modèle

In [67]:
pred = mnb.predict(X_test)

In [65]:
pred.shape

(6393,)

In [68]:
 y_test

13667    0
22090    0
21397    0
26091    0
23671    0
        ..
10840    0
2337     0
20627    0
15583    1
28627    0
Name: label, Length: 6393, dtype: int64

# Accuracy et matrice de confusion

In [70]:
fnl = pd.DataFrame(np.c_[y_test, pred], columns=['Actual','Predicted'])
fnl

,Actual,Predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
6388,0,0
6389,0,0
6390,0,0
6391,1,0


# Sauvegarder le modele

In [72]:
import joblib

In [73]:
joblib.dump(mnb, 'Twitter_Sentiment_Analysis.pkl')

['Twitter_Sentiment_Analysis.pkl']

# Charger le modele

In [86]:
model = joblib.load("Twitter_Sentiment_Analysis.pkl")

# Tester un modele

In [107]:
def test_model(test_sentence):                      #0 = spam et 1 = ham
    rev = re.sub("[^a-zA-Z]",' ', test_sentence)
    rev = rev.lower() 
    rev = rev.split()
    rev = [ps.stem(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = ' '.join(rev)
    
    rev = cv.transform([rev]).toarray()
    output = model.predict(rev)[0]
    
    if output == 0:
        print('Analyse positive')
    else:
        print('Analyse négative')

In [116]:
test_model("User see Goldern temple with condemn act")

Analyse négative
